Cryptocurrency trading bot.

Global variable definitions.

In [1]:
traded_asset_symbols = ['OMG', 'BTC', 'USDT']
intervals = ['1h', '1d']
period = '1 month ago UTC'

percentage_to_trade = 0.999999

api_key = 'yoyoyo'
api_secret = 'yeyeye'

Installs.

In [2]:
!pip install python-binance --upgrade --no-cache-dir
!pip install plotly stocktrends

Requirement already up-to-date: python-binance in ./anaconda3/lib/python3.7/site-packages (0.7.5)
You should consider upgrading via the '/home/samuel/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/samuel/anaconda3/bin/python -m pip install --upgrade pip' command.


Library imports.

In [3]:
%matplotlib inline

from binance.client import Client
from binance.enums import *
from binance.websockets import BinanceSocketManager
from binance.exceptions import BinanceAPIException
from twisted.internet import reactor
from stocktrends import Renko
from IPython import display

import math
import random
import time
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

Function definitions.

In [4]:
class Cryptocurrency_indicators:
    def print_OHLC(self, indicator_data):
        fig = go.Figure(data=[go.Candlestick(x=indicator_data.index.to_series(), 
                                             open=indicator_data['open'], 
                                             high=indicator_data['high'], 
                                             low=indicator_data['low'], 
                                             close=indicator_data['close'])])

        fig.show()

    def calculate_moving_average(self, dataset, window=2, method='simple'):
        if method == 'simple':
            dataset = dataset.rolling(window=window)
        elif method == 'exponential':
            dataset = dataset.ewm(span=window)

        dataset = dataset.mean()
        dataset = dataset.fillna(method='backfill', axis='index')
        return dataset

    def calculate_renko(self, dataset, brick_size=1):
        renko_ = dataset[['open', 'high', 'low', 'close']].copy().astype(float)
        renko_.index.name = 'date'
        renko_ = Renko(renko_.reset_index())
        renko_.brick_size = brick_size
        renko_ = renko_.get_ohlc_data().set_index('date')
        renko_.index.name = 'time'
        return renko_.uptrend

    def calculate_heikin_ashi(self, df):
        df = df.tz_localize(tz=None, ambiguous='infer')
        df = df[['open', 'high', 'low', 'close']].copy().astype('float')
        heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['open', 'high', 'low', 'close'])
        heikin_ashi_df['close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4

        for i in range(len(df)):
            if i == 0:
                heikin_ashi_df.iat[0, 0] = df['open'].iloc[0]
            else:
                heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i - 1, 0] + heikin_ashi_df.iat[i - 1, 3]) / 2

        heikin_ashi_df['high'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['high']).max(axis=1)
        heikin_ashi_df['low'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['low']).min(axis=1)

        return heikin_ashi_df

    def calculate_RSI(self, ticker, span=14):
        up = ticker['close'].astype(float).pct_change()
        down = up.copy()
        up[up < 0.0] = 0.0
        down[down > 0.0] = 0.0

        up = up.ewm(span=span).mean().fillna(method='pad')
        down = down.ewm(span=span).mean().abs().fillna(method='pad')

        ticker = up / down
        ticker = 100 - (100 / (1 + ticker))

        ticker = ticker.replace([-np.inf, np.inf], np.nan)
        ticker.iloc[0] = ticker.iloc[1]
        return ticker.fillna(method='pad')

    def calculate_MACD(self, ticker, span1=12, span2=26, average='exponential'):
        shorter = self.calculate_moving_average(ticker, window=span1, method=average)['close']
        longer = self.calculate_moving_average(ticker, window=span2, method=average)['close']

        ticker = (shorter - longer).replace([-np.inf, np.inf], np.nan)
        ticker.iloc[0] = ticker.iloc[1]
        ticker = ticker.fillna(method='pad').to_frame(name='MACD')
        ticker['signal_line'] = ticker.MACD.ewm(span=9).mean()
        ticker['histogram'] = ticker.MACD - ticker.signal_line
        return ticker

    def calculate_ATR(self, ticker, min_periods=14):
        ticker = ticker.astype(float)
        up = pd.DataFrame([ticker['high'].shift(), 
                           ticker['close']]).fillna(method='backfill', axis='columns').max(axis='index')

        down = pd.DataFrame([ticker['low'].shift(), 
                             ticker['close']]).fillna(method='backfill', axis='columns').min(axis='index')

        ticker = (up - down).ewm(alpha=1 / min_periods, 
                                 min_periods=min_periods, 
                                 adjust=False).mean()

        ticker = ticker.replace([-np.inf, np.inf], np.nan)
        return ticker.fillna(method='backfill')

    def calculate_bollinger_bands(self, dataframe, period=20):
        df = dataframe.copy()
        df['MA'] = self.calculate_moving_average(df, window=period, method='simple')['close']
        df['BB_up'] = df['MA'] + df['MA'].rolling(period).std()
        df['BB_down'] = df['MA'] - df['MA'].rolling(period).std()
        df['BB_width'] = df['BB_up'] - df['BB_down']
        return df[['BB_up', 'BB_down', 'BB_width']].dropna()

    def calculate_average_directional_index(self, df, n=14, n_ADX=14):
        dataset_index = df.index.copy()
        df = df.reset_index().drop(columns=['time']).astype(float)
        i = 0
        UpI = []
        DoI = []
        while i + 1 <= df.index[-1]:
            UpMove = df.loc[i + 1, 'high'] - df.loc[i, 'high']
            DoMove = df.loc[i, 'low'] - df.loc[i + 1, 'low']
            if UpMove > DoMove and UpMove > 0:
                UpD = UpMove
            else:
                UpD = 0
            UpI.append(UpD)
            if DoMove > UpMove and DoMove > 0:
                DoD = DoMove
            else:
                DoD = 0
            DoI.append(DoD)
            i += 1
        ATR = self.calculate_ATR(df, min_periods=14)
        UpI = pd.Series(UpI)
        DoI = pd.Series(DoI)
        PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean() / ATR, name='PosDI')
        NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean() / ATR, name='NegDI')
        ADX = pd.Series((abs(PosDI - NegDI) / (PosDI + NegDI)).ewm(span=n_ADX, min_periods=n_ADX).mean(), name='ADX')
        df = df.join(ADX).join(PosDI).join(NegDI)
        df.index = dataset_index
        return df[['ADX', 'PosDI', 'NegDI']].dropna()

    def calculate_commodity_channel_index(self, dataset, min_periods=20):
        PP = (dataset['high'] + dataset['low'] + dataset['close']) / 3
        CCI = pd.Series((PP - PP.rolling(min_periods, min_periods=min_periods).mean()) / \
                        PP.rolling(min_periods, min_periods=min_periods).std(),
                        name='CCI')
        return dataset.join(CCI)['CCI']

    def calculate_KDJ(self, dataset):
        def get_rsv(dataset):
            low_min = dataset['low'].rolling(min_periods=1, window=9, center=False).min()
            high_max = dataset['high'].rolling(min_periods=1, window=9, center=False).max()
            return ((dataset['close'] - low_min) / (high_max - low_min)).fillna(0).astype(float) * 100

        def calc_kd(column):
            k = 50.0
            for i in (1.0 / 3.0) * column:
                k = (2.0 / 3.0) * k + i
                yield k

        dataset['K'] = list(calc_kd(get_rsv(dataset)))
        dataset['D'] = list(calc_kd(dataset['K']))
        dataset['J'] = 3 * dataset['K'] - 2 * dataset['D']
        return dataset[['K', 'D', 'J']]

    def calculate_relative_volume_level(self, 
                                        dataset, 
                                        average_1=26, 
                                        average_2=14, 
                                        threshold=1.75, 
                                        method='exponential'):

        volume = dataset[['volume']]
        volume_average = self.calculate_moving_average(volume, window=average_1, method=method)
        relative_volume = volume / average_1
        smoothed_relative_volume = self.calculate_moving_average(relative_volume, window=average_2, method=method)
        relative_volume_level = smoothed_relative_volume.iloc[-1] / smoothed_relative_volume.iloc[-2]
        return relative_volume_level > threshold


class Cryptocurrency_pair_info:
    def __init__(self, client, pair):
        self.client = client
        self.pair = pair

        pair_info = client.get_symbol_info(self.pair)
        self.base_asset = pair_info['baseAsset']
        self.quote_asset = pair_info['quoteAsset']
        self.precision = pair_info['quotePrecision']

        filters = pair_info['filters']
        price_filter = [ticker for ticker in filters if ticker['filterType'] == 'PRICE_FILTER']
        lot_size = [ticker for ticker in filters if ticker['filterType'] == 'LOT_SIZE']
        self.tick_size = [ticker['tickSize'].find('1') - 2 for ticker in price_filter][0]
        self.step_size = [ticker['stepSize'].find('1') - 2 for ticker in lot_size][0]

        self.calculate_balance()

    def calculate_balance(self):
        self.base_asset_balance = float(self.client.get_asset_balance(asset=self.base_asset)['free'])
        self.quote_asset_balance = float(self.client.get_asset_balance(asset=self.quote_asset)['free'])
        self.pair_last_price = float(self.client.get_ticker(symbol=self.pair)['lastPrice'])
        self.pair_buy_balance = self.quote_asset_balance / self.pair_last_price
        self.pair_sell_balance = self.base_asset_balance * self.pair_last_price
        self.pair_combined_base_balance = self.pair_buy_balance + self.base_asset_balance
        self.pair_combined_quote_balance = self.pair_sell_balance + self.quote_asset_balance

    def print_balance(self):
        print('base_asset_balance: ', self.base_asset_balance)
        print('quote_asset_balance: ', self.quote_asset_balance)
        print('pair_last_price: ', self.pair_last_price)
        print('pair_buy_balance: ', self.pair_buy_balance)
        print('pair_sell_balance: ', self.pair_sell_balance)
        print('pair_combined_base_balance: ', self.pair_combined_base_balance)
        print('pair_combined_quote_balance: ', self.pair_combined_quote_balance)


class Cryptocurrency_triggers:
    def __init__(self, dataset, indicators):
        self.indicators = indicators
        self.dataset = dataset
        self.calculate_indicators()
        self.calculate_triggers()

    def calculate_indicators(self):
        dataset = self.dataset.astype(float)
        self.heikin_ashi = self.indicators.calculate_moving_average(dataset, window=1, method='simple')
        self.heikin_ashi = self.indicators.calculate_heikin_ashi(self.heikin_ashi)
        self.heikin_ashi = self.indicators.calculate_moving_average(dataset, window=1, method='exponential')
        self.RSI_6 = self.indicators.calculate_RSI(dataset, span=6)
        self.RSI_12 = self.indicators.calculate_RSI(dataset, span=12)
        self.CCI = self.indicators.calculate_commodity_channel_index(self.heikin_ashi, min_periods=20)
        self.brick_size = self.indicators.calculate_ATR(dataset, min_periods=14).median()
        self.renko = self.indicators.calculate_renko(dataset, brick_size=self.brick_size)
        self.ADX = self.indicators.calculate_average_directional_index(dataset, n=14, n_ADX=14)
        self.KDJ = self.indicators.calculate_KDJ(dataset)
        self.MACD = self.indicators.calculate_MACD(dataset)
        self.bollinger_bands = self.indicators.calculate_bollinger_bands(dataset, period=20)

    def calculate_triggers(self):
        dataset = self.dataset.astype(float)
        self.trend_trigger = (self.heikin_ashi['close'] - self.heikin_ashi['open']) > 0
        self.overtraded_trigger = self.RSI_6 > self.RSI_12
        self.trend_strength_trigger = self.ADX['ADX'] > 0.25
        self.trend_strength_positive_trigger = self.ADX['PosDI'] > self.ADX['NegDI']
        self.trend_strength_negative_trigger = self.ADX['PosDI'] < self.ADX['NegDI']
        self.momentum_trigger = self.KDJ['J'] > self.KDJ['D']
        self.volatility_trigger = (self.bollinger_bands['BB_width'] / dataset['close']) > 0.0005
        self.MACD_trigger = self.MACD['histogram'] > 0
        self.real_trigger = (dataset['close'] - dataset['open']) > 0


class Cryptocurrency_pair_at_interval:
    def __init__(self, client, indicators, pair_symbol, info, interval, period=period, download=True):
        self.client = client
        self.indicators = indicators
        self.pair_symbol = pair_symbol
        self.interval = interval
        self.period = period
        self.info = info
        self.dataset = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])

        if download:
            self.dataset = self.download_dataset(symbol=self.pair_symbol, 
                                                 interval=self.interval, 
                                                 period=self.period)

        self.triggers = Cryptocurrency_triggers(self.dataset, self.indicators)

    def download_dataset(self, symbol=None, interval=None, period=None):
        dataset_downloaded = \
            client.get_historical_klines(symbol=symbol, 
                                         interval=interval, 
                                         start_str=period)

        dataset_downloaded = pd.DataFrame(dataset_downloaded, 
                                          columns=['time', 
                                                   'open', 
                                                   'high', 
                                                   'low', 
                                                   'close', 
                                                   'volume', 
                                                   'Close time', 
                                                   'Quote asset volume', 
                                                   'Number of trades', 
                                                   'Taker buy base asset volume', 
                                                   'Taker buy quote asset volume', 
                                                   'Ignore'])

        four_hours = 14400
        milliseconds = 1000

        dataset_downloaded['time'] = \
            dataset_downloaded['time'].apply(lambda timestamp: \
                                             datetime.datetime.fromtimestamp((timestamp / \
                                                                              milliseconds) - \
                                                                             four_hours))

        dataset_downloaded = dataset_downloaded[['time', 
                                                 'open', 
                                                 'high', 
                                                 'low', 
                                                 'close', 
                                                 'volume']]

        dataset_downloaded.set_index('time', inplace=True)
        dataset_downloaded = dataset_downloaded.applymap(lambda entry: entry.rstrip('0').rstrip('.'))
        return dataset_downloaded


class Cryptocurrency_pair:
    def __init__(self, client, indicators, pair_symbol, intervals=intervals, period=period, raw_price=True, download=True):
        self.client = client
        self.indicators = indicators
        self.pair_symbol = pair_symbol
        self.intervals = intervals
        self.period = period
        self.raw_price = raw_price
        self.download = download
        self.info = self.get_pair_info()
        self.dataset = self.get_datasets()
        self.state = 'entry'

    def calculate_position(self):
        if self.info.base_asset_buy_balance > self.info.base_asset_sell_balance:
            self.position = 'sell'
        elif self.info.base_asset_sell_balance > self.info.base_asset_buy_balance:
            self.position = 'buy'

    def get_pair_info(self):
        return Cryptocurrency_pair_info(client=self.client, pair=self.pair_symbol).__dict__

    def get_datasets(self):
        dataset = dict()

        if self.raw_price:
            dataset['price'] = None

        for interval in self.intervals:
            dataset[interval] = Cryptocurrency_pair_at_interval(client=self.client, 
                                                                indicators=self.indicators, 
                                                                pair_symbol=self.pair_symbol, 
                                                                info=self.info, 
                                                                interval=interval, 
                                                                period=self.period, 
                                                                download=self.download)

        return dataset

    def make_OHLC_dataset(self, dataset, interval=intervals[0]):
        dataset = dataset.resample(interval)
        volume = dataset.agg({'quantity': ['sum']})['quantity']
        dataset = dataset.agg({'price': ['first', 'max', 'min', 'last']})['price']
        dataset['volume'] = volume
        dataset = dataset.dropna(axis='index')
        dataset = dataset.rename(columns={'first': 'open', 
                                          'max': 'high', 
                                          'min': 'low', 
                                          'last': 'close', 
                                          'sum': 'volume'})

        return dataset

    def trade(self):
        self.info.calculate_balance()

        if self.position == 'sell':
            coins_available = self.info.pair_buy_balance
            side = Client.SIDE_BUY
            position = 'buy'
        elif self.position == 'buy':
            coins_available = self.info.pair_sell_balance
            side = Client.SIDE_SELL
            position = 'sell'

        coins_available *= percentage_to_trade
        quantity = math.floor(coins_available * 10**self.info.step_size) / float(10**self.info.step_size)

        if self.info.tick_size < 0:
            quantity = math.floor(coins_available * abs(self.info.tick_size)) / float(abs(self.info.tick_size))

        client.create_order(symbol=self.pair_symbol, 
                            side=side, 
                            type=Client.ORDER_TYPE_MARKET, 
                            quantity=quantity, 
                            recvWindow=2000)

        self.info.calculate_balance()
        self.info.print_balance()

        if self.state == 'exit':
            self.state = 'entry'
        elif self.state == 'entry':
            self.state = 'exit'

        print("\nPosition for base asset " + self.info.base_asset + ' is ' + position + '.')
        print('Position for quote asset ' + self.info.quote_asset + ' is ' + self.position + '.')
        print('State is ' + self.state + ".\n")
        self.position = position


class Cryptocurrency_pairs:
    def __init__(self, 
                 client, 
                 traded_asset_symbols=traded_asset_symbols, 
                 intervals=intervals, 
                 raw_price=True):

        self.indicators = Cryptocurrency_indicators()
        self.all_symbols = self.get_all_symbols(client)

        traded_pair_symbols = self.get_traded_pairs_from_assets(traded_asset_symbols)
        self.pairs = self.get_pairs(client, intervals, raw_price, traded_pair_symbols)
        self.traded_pair_symbol = self.get_richest_asset(traded_asset_symbols, traded_pair_symbols)

    def get_all_symbols(self, client):
        return [symbol['symbol'] for symbol in client.get_all_tickers()]

    def get_traded_pairs_from_assets(self, traded_asset_symbols):
        traded_pair_symbols = []
        for base_asset in traded_asset_symbols:
            for quote_asset in traded_asset_symbols:
                if base_asset != quote_asset:
                    for pair in [base_asset + quote_asset, quote_asset + base_asset]:
                        if pair in self.all_symbols:
                            traded_pair_symbols.append(pair)

        return traded_pair_symbols

    def get_pairs(self, client, intervals, raw_price, traded_pair_symbols):
        pairs = dict()
        for pair_symbol in traded_pair_symbols:
            pairs[pair_symbol] = Cryptocurrency_pair(client=client, 
                                                     indicators=self.indicators, 
                                                     pair_symbol=pair_symbol, 
                                                     intervals=intervals, 
                                                     raw_price=raw_price)

        return pairs

    def get_richest_asset(self, traded_asset_symbols, traded_pair_symbols):
        richest_count = dict()
        for traded_asset_symbol in traded_asset_symbols:
            richest_count.update({traded_asset_symbol: 0})

        for traded_pair_symbol in traded_pair_symbols:
            base_asset = self.pairs[traded_pair_symbol].info['base_asset']
            quote_asset = self.pairs[traded_pair_symbol].info['quote_asset']
            if self.pairs[traded_pair_symbol].info['base_asset_balance'] >= self.pairs[traded_pair_symbol].info['quote_asset_balance']:
                richest_count.update({base_asset: richest_count[base_asset] + 1})
            elif self.pairs[traded_pair_symbol].info['base_asset_balance'] < self.pairs[traded_pair_symbol].info['quote_asset_balance']:
                richest_count.update({quote_asset: richest_count[quote_asset] + 1})

        return max(richest_count, key=richest_count.get)

In [5]:
client = Client(api_key, api_secret)

pairs = Cryptocurrency_pairs(client=client, 
                             traded_asset_symbols=traded_asset_symbols, 
                             intervals=intervals, 
                             raw_price=False)

pairs.__dict__.keys()

dict_keys(['indicators', 'all_symbols', 'pairs', 'traded_pair_symbol'])

In [6]:
pairs.pairs['BTCUSDT'].dataset['1d'].dataset

,open,high,low,close,volume
time,,,,,
2020-05-07 16:00:00,9986.3,10035.96,9705,9800.01,100683.7964
2020-05-08 16:00:00,9800.02,9914.25,9520,9539.4,81950.679567
2020-05-09 16:00:00,9539.1,9574.83,8117,8722.77,183865.182028
2020-05-10 16:00:00,8722.77,9168,8200,8561.52,168807.251832
2020-05-11 16:00:00,8562.04,8978.26,8528.78,8810.79,86522.780066
2020-05-12 16:00:00,8810.99,9398,8792.99,9309.37,92466.274018
2020-05-13 16:00:00,9309.35,9939,9256.76,9791.98,129565.37747
2020-05-14 16:00:00,9791.97,9845.62,9150,9316.42,115890.761516
2020-05-15 16:00:00,9315.96,9588,9220,9381.27,59587.627862


In [ ]:
# --- Development separation --- #

In [ ]:
for symbol in symbols.keys():
    symbols[symbol] = Cryptocurrency(client, symbol, interval)
    print(symbols[symbol].dataset)

In [ ]:
for symbol in symbols.keys():
    print(symbol)
    print(symbols[symbol].heikin_ashi)
    fig = go.Figure(data=[go.Candlestick(x=symbols[symbol].heikin_ashi.index.to_series(), 
                                         open=symbols[symbol].heikin_ashi['open'], 
                                         high=symbols[symbol].heikin_ashi['high'], 
                                         low=symbols[symbol].heikin_ashi['low'], 
                                         close=symbols[symbol].heikin_ashi['close'])])

    fig.show()

In [ ]:
for symbol in symbols.keys():
    print(symbols[symbol].CCI)
    plt.plot(symbols[symbol].CCI)
    plt.show()

In [ ]:
for symbol in symbols.keys():
    print(symbols[symbol].renko)
    plt.plot(symbols[symbol].renko)
    plt.show()

In [ ]:
for symbol in symbols.keys():
    print(symbols[symbol].ADX)
    plt.plot(symbols[symbol].ADX)
    plt.show()

In [ ]:
for symbol in symbols.keys():
    print(symbols[symbol].KDJ)
    plt.plot(symbols[symbol].KDJ)
    plt.show()

In [ ]:
for symbol in symbols.keys():
    print(symbols[symbol].MACD)
    plt.plot(symbols[symbol].MACD)
    plt.show()

In [ ]:
for symbol in symbols.keys():
    print(symbols[symbol].bollinger_bands)
    plt.plot(symbols[symbol].bollinger_bands)
    plt.show()

In [ ]:
'''
def process_message(msg):
    global symbols, traded_symbol
    four_hours = 14400
    milliseconds = 1000

    msg = msg['data']['k']
    symbol = msg['s']
    dataset = symbols[symbol].dataset
    quote_asset_precision = symbols[symbol].quote_asset_precision
    if symbol in symbols.keys():
        last_data = \
            pd.DataFrame([["{:0.0{}f}".format(float(msg['o']), quote_asset_precision).rstrip('0').rstrip('.'), 
                           "{:0.0{}f}".format(float(msg['h']), quote_asset_precision).rstrip('0').rstrip('.'), 
                           "{:0.0{}f}".format(float(msg['l']), quote_asset_precision).rstrip('0').rstrip('.'), 
                           "{:0.0{}f}".format(float(msg['c']), quote_asset_precision).rstrip('0').rstrip('.'), 
                           "{:0.0{}f}".format(float(msg['v']), quote_asset_precision).rstrip('0').rstrip('.')]], 
                         index=[datetime.datetime.fromtimestamp(int(msg['t'] / milliseconds) - four_hours)], 
                         columns=['open', 'high', 'low', 'close', 'volume'])

        last_data.index.name = 'time'

        if dataset.index[-1] == last_data.index[-1]:
            dataset = dataset.iloc[:-1].append(last_data).iloc[-1000:]
        else:
            dataset = dataset.append(last_data).iloc[-1000:]

        symbols[symbol].dataset = dataset
        symbols[symbol].calculate_indicators()

        if symbol == traded_symbol:
            if symbols[symbol].state == 'exit':

                if symbols[symbol].position == 'sell' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        symbols[symbol].MACD_trigger.iloc[-1] and \
                        symbols[symbol].real_trigger.iloc[-1] and \
                        symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

                elif symbols[symbol].position == 'buy' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        not symbols[symbol].MACD_trigger.iloc[-1] and \
                        not symbols[symbol].real_trigger.iloc[-1] and \
                        not symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

            elif symbols[symbol].state == 'entry':

                if symbols[symbol].position == 'sell' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        symbols[symbol].MACD_trigger.iloc[-1] and \
                        symbols[symbol].real_trigger.iloc[-1] and \
                        symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

                elif symbols[symbol].position == 'buy' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        not symbols[symbol].MACD_trigger.iloc[-1] and \
                        not symbols[symbol].real_trigger.iloc[-1] and \
                        not symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

bm = BinanceSocketManager(client, user_timeout=43200)
conn_key = bm.start_multiplex_socket([symbol.lower() + '@kline_' + interval for symbol in symbols.keys()], 
                                     process_message)
bm.start()
'''

datasets = dict()
for symbol in symbols.keys():
    datasets.update({symbol: {'price': None, 
                              '15s': pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume']), 
                              '1m': pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])}})

def process_message(msg):
    global datasets, traded_symbol, intervals
    four_hours = 14400
    millisecond = 1000

    msg = msg['data']
    symbol = msg['s']
    dataset = datasets[symbol]
    price = dataset['price']
    quote_asset_precision = symbols[symbol].quote_asset_precision

    last_data = \
        pd.DataFrame([["{:0.0{}f}".format(float(msg['a']), quote_asset_precision).rstrip('0').rstrip('.'), 
                       "{:0.0{}f}".format(float(msg['p']), quote_asset_precision).rstrip('0').rstrip('.'), 
                       "{:0.0{}f}".format(float(msg['q']), quote_asset_precision).rstrip('0').rstrip('.')]], 
                     index=[datetime.datetime.fromtimestamp(int(msg['E'] / millisecond) - four_hours)], 
                     columns=['id', 'price', 'quantity'])

    last_data.index.name = 'time'

    if price is None:
        dataset.update({'price': last_data})
    elif price['id'].iloc[-1] > last_data['id'].iloc[-1]:
        price = price.append(last_data).iloc[-1000:]

    for interval in intervals:
        dataset[interval] = make_OHLC_dataset(price, interval=interval)

    dataset['price'] = price
    datasets[symbol] = dataset
    datasets[symbol].calculate_indicators()

    if symbol == traded_symbol:
        if symbols[symbol].state == 'exit':

                if symbols[symbol].position == 'sell' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        symbols[symbol].MACD_trigger.iloc[-1] and \
                        symbols[symbol].real_trigger.iloc[-1] and \
                        symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

                elif symbols[symbol].position == 'buy' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        not symbols[symbol].MACD_trigger.iloc[-1] and \
                        not symbols[symbol].real_trigger.iloc[-1] and \
                        not symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

            elif symbols[symbol].state == 'entry':

                if symbols[symbol].position == 'sell' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        symbols[symbol].MACD_trigger.iloc[-1] and \
                        symbols[symbol].real_trigger.iloc[-1] and \
                        symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

                elif symbols[symbol].position == 'buy' and \
                        symbols[symbol].volatility_trigger.iloc[-1] and \
                        not symbols[symbol].MACD_trigger.iloc[-1] and \
                        not symbols[symbol].real_trigger.iloc[-1] and \
                        not symbols[symbol].momentum_trigger.iloc[-1]:

                    symbols[symbol].trade()

bm = BinanceSocketManager(client, user_timeout=43200)
conn_key = bm.start_multiplex_socket([symbol.lower() + '@trade' for symbol in symbols.keys()], 
                                     process_message)
bm.start()

'''
scores = {}
for crypto_pair in symbols.values():
    scores.update({crypto_pair.base_asset: 0})
    scores.update({crypto_pair.quote_asset: 0})

while True:
    time.sleep(15)
    for symbol in symbols.keys():
        symbols[symbol].calculate_balance()
        symbols[symbol].calculate_position()

    scores = {}
    for crypto_pair in symbols.values():
        scores.update({crypto_pair.base_asset: 0})
        scores.update({crypto_pair.quote_asset: 0})

    for asset in scores.keys():
        for symbol in symbols.keys():
            if symbols[symbol].base_asset == asset:
                if symbols[symbol].real_trigger.iloc[-1] and \
                        symbols[symbol].real_trigger_10m.iloc[-1]:
                    scores.update({asset: scores[asset] + 1})
                else:
                    scores.update({asset: scores[asset] - 1})
            elif symbols[symbol].quote_asset == asset:
                if symbols[symbol].real_trigger.iloc[-1] and \
                        symbols[symbol].real_trigger_10m.iloc[-1]:
                    scores.update({asset: scores[asset] - 1})
                else:
                    scores.update({asset: scores[asset] + 1})

    assets = scores.keys()
    best_assets = [asset for asset in scores.keys() if scores[asset] > 0]
    print('best_assets: ', best_assets)

    if best_assets != [] and \
            not (symbols[traded_symbol].base_asset in best_assets or \
            symbols[traded_symbol].quote_asset in best_assets):

        if symbols[traded_symbol].position == 'buy':
            asset = symbols[traded_symbol].base_asset
        elif symbols[traded_symbol].position == 'sell':
            asset = symbols[traded_symbol].quote_asset

        target_symbols = [symbol for symbol in symbols.keys() \
                          if asset in symbol and any([asset in symbol for asset in best_assets])]

        target_symbol = target_symbols[0]
        symbols[target_symbol].state = 'exit'

        if symbols[target_symbol].base_asset == asset:
            symbols[target_symbol].position = 'buy'
        elif symbols[target_symbol].quote_asset == asset:
            symbols[target_symbol].position = 'sell'

        traded_symbol = target_symbol
'''

In [ ]:
#bm.stop_socket(conn_key)
#bm.close()
#reactor.stop()